In [1]:
import pickle

In [2]:
vectorizer_path = 'vectorizer.pickle'
vectorizer = pickle.load(
    open(vectorizer_path, 'rb'))

gb_path = 'gb.pickle'
gboost_m = pickle.load(
    open(gb_path, 'rb'))

transformer_path = 'transformer.pickle'
transformer = pickle.load(
    open(transformer_path, 'rb'))


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec
import re

In [4]:
import string

def data_cleaning(data):
    cleaned_data = []
    fillerWord = ("so","yeah","okay","um","uh","mmm","ahan","uh","huh","ahm","oh","sooo","uh","huh","yeh","yah","hmm","bye")
    fillerword_reg= "bye[.,]|so[.,]|yeah[.,]|okay[.,]|um[.,]|uh[.,]|mmm[.,]|ahan[.,]|uh[.,]|huh[.,]|ahm[.,]|oh[.,]|sooo[.,]|uh[.,]|huh[.,]|yeh[.,]|yah[.,]|hmm[.,]"
    STOPWORDS = set(stopwords.words('english'))
    remove=["doesn't","not","nor","neither","isn't","hadn't","mightn't","needn't","wasn't"]
    for i in remove:
        STOPWORDS.discard(i)
    
    STOPWORDS.add(fillerWord)  
    for i in range(len(data)):
        intent = re.sub("#", "", data[i])#extracting hashtags
        intent = re.sub(r'^https?:\/\/.*[\r\n]*', '',intent, flags=re.MULTILINE)#extracting links
        html=re.compile(r'<.*?>')#extracting html tags
        intent =html.sub(r"", intent)
        #extracting symbols and characters
        intent=re.sub(r'@\w+',"",intent)
        intent=re.sub(r'#\w+',"",intent) 
        intent=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', intent) 
        punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
        intent.rstrip(string.punctuation)
        intent=re.sub('[^A-Za-z\s]+',"", intent)
        intent = intent.lower()
        intent = intent.split()
        #Lemmatization to normalise text
        intent = [lemmatizer.lemmatize(word) for word in intent if not word in STOPWORDS]
        intent = ' '.join(intent)
        filler=re.compile(fillerword_reg)
        intent=filler.sub("",intent)
        cleaned_data.append(intent)
    return cleaned_data

In [5]:
def intent_classification(intent):
    
    ####Preprocessing and cleaning train data
    process_text=data_cleaning(intent)       
    intent_vector = vectorizer.transform([process_text[0]])
    
    
    #print(intent_vector)
    intent_vector_data = transformer.transform(intent_vector).toarray()
    predicted_class = gboost_m.predict(intent_vector_data)[0]
    d={"12" :"uniinfo",
    "1" :"courseinfo",
    "4" :"facultyinfo",
    "11":"staffinfo",
    "7":"hostelinfo",
    "8":"placesinfo",
    "0" :"careerinfo ",
    "3" :"eventinfo" ,
    "10":"sportinfo",
    "2":"entryinfo",
    "9":"regevent",
    "5":"fillhostelapplication",
    "6":"greeting"}
    return d[str(predicted_class)]
    

In [25]:
print(intent_classification(['say tell me entry ']))

staffinfo
